<a href="https://colab.research.google.com/github/joseorlandomx/topicodeindustria1/blob/main/Practica_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>